In [1]:
!conda info


     active environment : base
    active env location : /lfs/emanuelezappala.isu/miniconda3
            shell level : 1
       user config file : /lfs/emanuelezappala.isu/.condarc
 populated config files : 
          conda version : 23.3.1
    conda-build version : not installed
         python version : 3.8.16.final.0
       virtual packages : __archspec=1=x86_64
                          __cuda=12.4=0
                          __glibc=2.28=0
                          __linux=4.18.0=0
                          __unix=0=0
       base environment : /lfs/emanuelezappala.isu/miniconda3  (writable)
      conda av data dir : /lfs/emanuelezappala.isu/miniconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package c

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import logging
import warnings
from typing import Callable, Optional, Union
import os, argparse

import numpy as np
import torch

from scipy import integrate
import time
import pickle
from torch.utils.data import Dataset

logger = logging.getLogger("iesolver")
logger.setLevel(logging.WARNING)#(logging.DEBUG)

import matplotlib.pyplot as plt


import torchcubicspline
from torchcubicspline import(natural_cubic_spline_coeffs, 
                             NaturalCubicSpline)
                             
#from torchdiffeq import odeint


from source.integrators import MonteCarlo 
mc = MonteCarlo()

from source.model import F_NN, basis, Leray_Schauder, Leray_Schauder_model, interpolated_func, Simple_NN

from source.utils import fix_random_seeds,to_np

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"
   
    
parser = argparse.ArgumentParser(description='Leray_Schauder')
parser.add_argument('-root_path', metavar='DIR', default='',
                    help='path to dataset')
parser.add_argument('-dataset-name', default='stl10',
                    help='dataset name', choices=['acrobot_dataset'])

parser.add_argument('-j', '--workers', default=12, type=int, metavar='N',
                    help='number of data loading workers (default: 32)')
parser.add_argument('--epochs', default=3000, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('-b', '--batch_size', default=20, type=int,
                    metavar='N',
                    help='mini-batch size (default: 256), this is the total '
                         'batch size of all GPUs on the current node when '
                         'using Data Parallel or Distributed Data Parallel')
parser.add_argument('--lr', '--learning-rate', default=1e-4, type=float,
                    metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--wd', '--weight-decay', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)',
                    dest='weight_decay')
parser.add_argument('--seed', default=None, type=int,
                    help='seed for initializing training. ')
parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--fp16-precision', action='store_true',
                    help='Whether or not to use 16-bit precision GPU training.')

parser.add_argument('--out_dim', default=128, type=int,
                    help='feature dimension (default: 128)')
parser.add_argument('--log-every-n-steps', default=100, type=int,
                    help='Log every n steps')
parser.add_argument('--temperature', default=0.07, type=float,
                    help='softmax temperature (default: 0.07)')
parser.add_argument('--n-views', default=2, type=int, metavar='N',
                    help='Number of views for contrastive learning training.')
parser.add_argument('--gpu-index', default=0, type=int, help='Gpu index.')
parser.add_argument('--model', default='simclr', choices=['simclr','lipschitz_simclr','vae','gan'], 
                    help='Models to be used')
parser.add_argument('--mode', default='train', choices=['train','evaluate'], 
                    help='Set to ''evaluate'' if inference is desired')
parser.add_argument('--training_split', default=0.25,type=float, 
                    help='Fraction of the samples that will be used for validation')
parser.add_argument('--resume_from_checkpoint', default=None, 
                    help='Give string to run number. Ex: "run12"')
parser.add_argument('--plot_freq', default=1, type=int,help='')
parser.add_argument('--experiment_name', default=None,help='')
    
    

_StoreAction(option_strings=['--experiment_name'], dest='experiment_name', nargs=None, const=None, default=None, type=None, choices=None, required=False, help='', metavar=None)

In [4]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F
from torch.autograd import Variable

from source.experiments import experiment

use_cuda = torch.cuda.is_available()

In [5]:
use_cuda

True

In [6]:
from functools import reduce
import random
import pickle
import time

In [7]:
args = parser.parse_args("")
args.model='Leray_Schauder'
args.mode='train'
args.mode='evaluate'
args.dataset_name = 'integral_equations'
args.seed = 7
args.experiment_name = 'IE_spirals'
args.plot_freq = 50
args.device = device
args.num_dim_plot = 2
args.lr = 1e-3
args.min_lr=1e-7
args.T_max = 101
args.plat_patience = 10
args.factor = 0.1
# args.temperature=0.001
#args.lr_scheduler = 'ReduceLROnPlateau'
args.lr_scheduler = 'CosineAnnealingLR'
args.resume_from_checkpoint = 'run64' #64,63 (64 no interpolation, 63 interpolation)
fix_random_seeds(args.seed)
args.perturbation_to_obs0=None
args.training_split=0.2
args.smoothing_factor= .5
if args.mode == 'train':
    args.n_batch = 16
else:
    args.n_batch = 1
args.n_basis = 8 #8
args.patience =  200
args.support_tensors=False
args.support_test=False
args.dim = 1
args.time_points = 100
#args.f_nn = True
args.max_iterations = 3
args.interpolation = False
args.channels = 16 #16
if args.mode == 'train':
    args.N_MC = 1000
else:
    args.N_MC = 5000
if args.interpolation and args.mode=='train':
    args.downsample = 25
else:
    args.downsample = 50
args.plot_train = False

In [8]:
print(args.downsample)

50


In [9]:
Data = pickle.load(open("50_IE_Spirals.pkl","rb"))
Data = Data.to(device).detach()

In [10]:
if args.mode == 'train':
    Data = Data[:-96,...].to(device)
if args.mode == 'evaluate':
    Data = Data[-96:,...].to(device)

In [11]:
if args.interpolation and args.mode=='train':
    print("Interpolation is on")
    Data = Data[:,::2,:]

In [12]:
Data.shape

torch.Size([96, 100, 2])

In [13]:
args.ts_integration = None

In [14]:
times = torch.linspace(-1,1,100).to(device)

In [15]:
if args.interpolation and args.mode=='train':
    print("Interpolation is on")
    times = times[::2]

In [16]:
times.shape

torch.Size([100])

In [17]:
epsilon = 2. #2.

In [18]:
Basis = basis(args.dim,args.channels,n=args.n_basis,shapes=[100,100,100],batch_size=args.n_batch).to(device)#[100,100,100]

In [19]:
LS = Leray_Schauder(Basis,dim=args.dim,channels=args.channels,epsilon=epsilon,N=args.N_MC,batch_size=args.n_batch,softmax=False).to(device)

/lfs/emanuelezappala.isu/miniconda3/envs/Spectr/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352645774/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [20]:
proj_operator = F_NN(args.n_basis*args.channels,args.n_basis*args.channels, [100,100]).to(device)#[100,100]

In [21]:
model = Leray_Schauder_model(LS,proj_operator,batch_size=args.n_batch).to(device)

In [22]:
args.decoder = Simple_NN(args.channels,2,[100,100]).to(device)

In [23]:
p_shapes = []
flat_parameters = []
for p in model.parameters():
    p_shapes.append(p.size())
    flat_parameters.append(p.flatten())
all_parameters = torch.cat(flat_parameters)

In [24]:
all_parameters.shape

torch.Size([212056])

In [25]:
p_shapes = []
flat_parameters = []
for p in args.decoder.parameters():
    p_shapes.append(p.size())
    flat_parameters.append(p.flatten())
all_parameters = torch.cat(flat_parameters)

In [26]:
times.shape

torch.Size([100])

In [27]:
experiment(model, Data, times, args)

Loading  Leray_Schauder/IE_spirals/run64/model
Running in evaluation mode


  0%|          | 0/96 [00:00<?, ?it/s]

/lfs/emanuelezappala.isu/miniconda3/envs/Spectr/lib/python3.10/site-packages/torchcubicspline/interpolate.py:277: UserWarning: torch.searchsorted(): boundary tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous boundary tensor if possible. This message will only appear once per program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352645774/work/aten/src/ATen/native/BucketizationUtils.h:39.)
  index = torch.bucketize(t.detach(), self._t) - 1


(0.0010649741255595775, tensor(0.0005))